In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from zipfile import ZipFile
from PIL import Image

In [ ]:
with ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zip_ref:
    zip_ref.extractall('.')
with ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
os.mkdir('/kaggle/working/dataset')

os.mkdir('/kaggle/working/dataset/training')
os.mkdir('/kaggle/working/dataset/validation')

os.mkdir('/kaggle/working/dataset/training/cats')
os.mkdir('/kaggle/working/dataset/training/dogs')
os.mkdir('/kaggle/working/dataset/validation/cats')
os.mkdir('/kaggle/working/dataset/validation/dogs')

In [ ]:
os.listdir('/kaggle/working/')

In [ ]:
filenames = os.listdir("./train")
categories = []
for name in filenames:
    category = name.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
        
df = pd.DataFrame({
    'filename' : filenames,
    'category' : categories
})

In [ ]:
df.head(10)

In [ ]:
df.category.value_counts().plot.bar()
plt.xticks([0,1],['Cats','Dogs'])
plt.title('Cats vs Dogs')
plt.ylabel('Number of Pictures')
plt.show()

In [ ]:
cats = pd.DataFrame(df[df.category == 0])
dogs = pd.DataFrame(df[df.category == 1])

In [ ]:
train_cats, val_cats = train_test_split(cats, test_size = 0.2, random_state = 42)
train_dogs, val_dogs = train_test_split(dogs, test_size = 0.2, random_state = 42)

In [ ]:
print(f'Number of files in train_cats : {train_cats.shape[0]}')
print(f'Number of files in val_cats: {val_cats.shape[0]}')
print(f'Number of files in train_dogs : {train_dogs.shape[0]}')
print(f'Number of files in val_dogs : {val_dogs.shape[0]}')

In [ ]:
for filename in train_cats.filename:
    shutil.copy(os.path.join('./train', filename),
                os.path.join('./dataset/training/cats', filename))
for filename in train_dogs.filename:
    shutil.copy(os.path.join('./train', filename), 
                os.path.join('./dataset/training/dogs', filename))
    
for filename in val_cats.filename:
    shutil.copy(os.path.join('./train', filename), 
                os.path.join('./dataset/validation/cats', filename))
for filename in val_dogs.filename:
    shutil.copy(os.path.join('./train', filename), 
                os.path.join('./dataset/validation/dogs', filename))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest'
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
)

In [ ]:
train_generator = train_datagen.flow_from_directory('./dataset/training',
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   target_size=(128,128))

val_generator = train_datagen.flow_from_directory('./dataset/validation',
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   target_size=(128,128))

In [ ]:
model = Sequential()

model.add(Conv2D(16,(3,3), activation = 'relu', input_shape = (128,128,3)))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), activation = 'relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=313,
      epochs=15, 
      validation_data=val_generator, 
      validation_steps=79,  
      verbose=2,
      )

In [ ]:
model.save_weights("model.h5")

In [ ]:
#os.listdir('./')

In [ ]:
#!rm -rf ./dataset